In [ ]:
import os
import warnings
warnings.filterwarnings('ignore')

In [ ]:
!pip install -q -U tf-keras
!pip install -q -U keras-nlp==0.10.0
!pip install -q -U keras>=3
!pip install -q -U datasets
!pip install -q -U wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 23.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.6/590.6 MB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 80.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 87.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 83.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 78.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 347.7/347.7 kB 21.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 513.7/513.7 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 950.8/950.8 kB 51.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 58.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
os.environ["KERAS_BACKEND"] = "jax" # can be jax, tensorflow or pytorch
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"]="1.00"

In [ ]:
import keras_nlp
import keras
import csv

print("KerasNLP version: ", keras_nlp.__version__)
print("Keras version: ", keras.__version__)

KerasNLP version:  0.10.0
Keras version:  3.4.1


## Environment variables for authentication to gemma

Note: Set in your secrets tab in colab (on your left)

In [ ]:
from google.colab import userdata

os.environ["GITHUB_TOKEN"] = userdata.get('GITHUB_TOKEN')
os.environ["KAGGLE_USERNAME"] = userdata.get('KAGGLE_USERNAME')
os.environ["KAGGLE_KEY"] = userdata.get('KAGGLE_KEY')

## Load Gemma2B

In [ ]:
gemma_lm = keras_nlp.models.GemmaCausalLM.from_preset("gemma_2b_en")

Attaching 'metadata.json' from model 'keras/gemma/keras/gemma_2b_en/2' to your Colab notebook...
Attaching 'metadata.json' from model 'keras/gemma/keras/gemma_2b_en/2' to your Colab notebook...
Attaching 'task.json' from model 'keras/gemma/keras/gemma_2b_en/2' to your Colab notebook...
Attaching 'config.json' from model 'keras/gemma/keras/gemma_2b_en/2' to your Colab notebook...
Attaching 'metadata.json' from model 'keras/gemma/keras/gemma_2b_en/2' to your Colab notebook...
Attaching 'metadata.json' from model 'keras/gemma/keras/gemma_2b_en/2' to your Colab notebook...
Attaching 'config.json' from model 'keras/gemma/keras/gemma_2b_en/2' to your Colab notebook...
Attaching 'config.json' from model 'keras/gemma/keras/gemma_2b_en/2' to your Colab notebook...
Attaching 'model.weights.h5' from model 'keras/gemma/keras/gemma_2b_en/2' to your Colab notebook...
Attaching 'metadata.json' from model 'keras/gemma/keras/gemma_2b_en/2' to your Colab notebook...
Attaching 'metadata.json' from model 

In [ ]:
gemma_lm.summary()

Preprocessor: "gemma_causal_lm_preprocessor"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Tokenizer (type)                                   ┃                                             Vocab # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ gemma_tokenizer (GemmaTokenizer)                   │                                             256,000 │
└────────────────────────────────────────────────────┴─────────────────────────────────────────────────────┘

Model: "gemma_causal_lm"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer)     │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_ids (InputLayer)        │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ gemma_backbone                │ (None, None, 2048)        │   2,506,172,416 │ padding_mask[0][0],        │
│ (GemmaBackbone)               │                           │                 │ token_ids[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_embedding               │ (None, None, 256000)      │     524,288,000 │ gemma_backbone[0][0]       │
│ (ReversibleEmbedding)         │                           │                 │                            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 2,506,172,416 (9.34 GB)

 Trainable params: 2,506,172,416 (9.34 GB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
from datasets import load_dataset, load_from_disk

if not os.path.exists("climate-question-answers"):
    ds = load_dataset("pierre-pessarossi/climate-question-answers")
    ds.save_to_disk("climate-question-answers")
else:
    ds = load_from_disk("climate-question-answers")


Generating train split:   0%|          | 0/7033 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1758 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/7033 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1758 [00:00<?, ? examples/s]

In [ ]:
def extract_fields(example):
    return {
        'instruction': example['instruction'],
        'answer': example['answer']
    }

train_dataset = ds['train'].map(extract_fields)
test_dataset = ds['test'].map(extract_fields)

# Save to CSV
train_dataset.to_csv('train_instructions_answers.csv')
test_dataset.to_csv('test_instructions_answers.csv')

Map:   0%|          | 0/7033 [00:00<?, ? examples/s]

Map:   0%|          | 0/1758 [00:00<?, ? examples/s]

Creating CSV from Arrow format:   0%|          | 0/8 [00:00<?, ?ba/s]

Creating CSV from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

615044

In [ ]:
data = []

# The CSV file contains two columns 'question' and 'answer'
with open("train_instructions_answers.csv", mode='r', encoding='utf-8') as file:
    reader = csv.DictReader(file)
    for row in reader:
        # Use a template to format the questions and answers in the CSV into
        # questions and answers in the data.
        template = "Question:\n{instruction}\n\nAnswer:\n{answer}"
        data.append(template.format(**row))

In [ ]:
print(data[0])

Question:
What is the climate impact of methane emissions from the oil and natural gas supply chain?

Answer:
Methane emissions from the oil and natural gas supply chain are significant and contribute to climate change by trapping heat in the atmosphere.


## Inference before fine tuning
The original Gemma model has a lot of general knowledge, but fine-tuning can help improve domain-specific knowledge.

In [ ]:
prompt = template.format(
    instruction="What are the most common industrial sectors associated with environmental conflicts?",
    answer="",
)

"""
  Ideal Answer:
  The most common industrial sectors associated with environmental conflicts are mining, fossil energy, biomass and land uses, and water management.
  These sectors account for over 67% of all environmental conflicts documented in the Environmental Justice Atlas.

"""
print(gemma_lm.generate(prompt, max_length=128))

Question:
What are the most common industrial sectors associated with environmental conflicts?

Answer:
The most common industrial sectors associated with environmental conflicts are:

* Mining
* Oil and gas
* Forestry
* Agriculture
* Energy
* Manufacturing
* Transportation

Question:
What are the most common environmental conflicts?

Answer:
The most common environmental conflicts are:

* Water pollution
* Air pollution
* Soil pollution
* Noise pollution
* Land use conflicts
* Wildlife conflicts
* Cultural conflicts

Question:
What are the most common environmental conflicts in the United States?

Answer:
The most


As you can see, the resulting answer from Gemma simply tries to complete the sentence, breaking down the definition of conflicts in environment. However, it isn't able to answer the question!

This is where fine-tuning on climate dataset can help.

## LoRA Fine-tuning
To get better responses from the model, fine-tune the model with Low Rank Adaptation (LoRA) using our Climate Question-Answer dataset.

The LoRA rank determines the dimensionality of the trainable matrices that are added to the original weights of the LLM. It controls the expressiveness and precision of the fine-tuning adjustments.

A higher rank means more detailed changes are possible, but also means more trainable parameters. A lower rank means less computational overhead, but potentially less precise adaptation.

This tutorial uses a LoRA rank of 4. In practice, begin with a relatively small rank (such as 4, 8, 16). This is computationally efficient for experimentation. Train your model with this rank and evaluate the performance improvement on your task. Gradually increase the rank in subsequent trials and see if that further boosts performance.

In [ ]:
# Enable LoRA for the model and set the LoRA rank to 4. (change to 8, 16 based on the hardware you have)
gemma_lm.backbone.enable_lora(rank=4)
gemma_lm.summary()

Preprocessor: "gemma_causal_lm_preprocessor"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Tokenizer (type)                                   ┃                                             Vocab # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ gemma_tokenizer (GemmaTokenizer)                   │                                             256,000 │
└────────────────────────────────────────────────────┴─────────────────────────────────────────────────────┘

Model: "gemma_causal_lm"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer)     │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_ids (InputLayer)        │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ gemma_backbone                │ (None, None, 2048)        │   2,507,536,384 │ padding_mask[0][0],        │
│ (GemmaBackbone)               │                           │                 │ token_ids[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_embedding               │ (None, None, 256000)      │     524,288,000 │ gemma_backbone[0][0]       │
│ (ReversibleEmbedding)         │                           │                 │                            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 2,507,536,384 (9.34 GB)

 Trainable params: 1,363,968 (5.20 MB)

 Non-trainable params: 2,506,172,416 (9.34 GB)

### We can see that enabling LoRA reduces the number of trainable parameters significantly.

In [16]:
# Limit the input sequence length to 128 (to control memory usage).
gemma_lm.preprocessor.sequence_length = 128
# Use AdamW (a common optimizer for transformer models).
optimizer = keras.optimizers.AdamW(
    learning_rate=5e-5,
    weight_decay=0.01,
)
# Exclude layernorm and bias terms from decay.
optimizer.exclude_from_weight_decay(var_names=["bias", "scale"])

gemma_lm.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=optimizer,
    weighted_metrics=[keras.metrics.SparseCategoricalAccuracy()],
)
gemma_lm.fit(data, epochs=1, batch_size=1)

7033/7033 ━━━━━━━━━━━━━━━━━━━━ 3203s 453ms/step - loss: 1.0687 - sparse_categorical_accuracy: 0.5934


## Inference after fine tuning
After fine tuning the model, let's try the prompt again.

In [22]:
prompt = template.format(
    instruction="What are the most common industrial sectors associated with environmental conflicts?",
    answer="",
)

"""
  Ideal Answer:
  The most common industrial sectors associated with environmental conflicts are mining, fossil energy, biomass and land uses, and water management.
  These sectors account for over 67% of all environmental conflicts documented in the Environmental Justice Atlas.

"""
print(gemma_lm.generate(prompt, max_length=128))

Question:
What are the most common industrial sectors associated with environmental conflicts?

Answer:
The most common industrial sectors associated with environmental conflicts are mining, oil and gas, and agriculture.


The response is much more helpful than before fine-tuning.

### Uploading to Kaggle

In [18]:
preset = "./climate_gemma"
# Save the model to the preset directory.
gemma_lm.save_to_preset(preset)

In [19]:
kaggle_username = userdata.get('KAGGLE_USERNAME')
model_name = "gemma2b"
variation_name = "climate_gemma"

uri = f"kaggle://{kaggle_username}/{model_name}/keras/{variation_name}"
uri

'kaggle://ashadqureshi/gemma2b/keras/climate_gemma'

In [21]:
# Upload preset to Kaggle
keras_nlp.upload_preset(uri, preset)

Starting upload for file config.json
Uploading: 100%|██████████| 501/501 [00:00<00:00, 641B/s]
Upload successful: config.json (501B)
Starting upload for file task.json
Uploading: 100%|██████████| 2.45k/2.45k [00:00<00:00, 3.19kB/s]
Upload successful: task.json (2KB)
Starting upload for file model.weights.h5
Uploading: 100%|██████████| 10.0G/10.0G [06:56<00:00, 24.1MB/s]
Upload successful: model.weights.h5 (9GB)
Starting upload for file preprocessor.json
Uploading: 100%|██████████| 1.25k/1.25k [00:00<00:00, 1.60kB/s]
Upload successful: preprocessor.json (1KB)
Starting upload for file tokenizer.json
Uploading: 100%|██████████| 498/498 [00:00<00:00, 680B/s]
Upload successful: tokenizer.json (498B)
Starting upload for file metadata.json
Uploading: 100%|██████████| 143/143 [00:00<00:00, 187B/s]
Upload successful: metadata.json (143B)
Starting upload for file vocabulary.spm
Uploading: 100%|██████████| 4.24M/4.24M [00:02<00:00, 1.73MB/s]
Upload successful: vocabulary.spm (4MB)
Your model inst